# Классификация на эмбеддингах

**Задача:** Обучить модель логистической регрессии на эмбеддингах. Напечатайть на экране значение accuracy на обучающей выборке.

Чтобы не создавать эмбеддинги слишком долго, отобрать из выборки только 400 случайных элементов. Для корректного тестирования поделите их на обучающую и тестовую выборки в соотношении 50:50.
Целевой признак находится в переменной `df_tweets['positive']`.

In [1]:
# импорт основных библиотек
import torch
import transformers
import numpy as np
import pandas as pd

# импорт спец. элементов
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
# чтение файла
df_tweets = pd.read_csv('/datasets/tweets.csv')

# выдержка из 400 случайных объектов
df_tweets = df_tweets.sample(400).reset_index(drop=True)

In [3]:
# инициализация токенизатора
tokenizer = transformers.BertTokenizer(vocab_file='/datasets/ds_bert/vocab.txt')

# токенизация
tokenized = df_tweets['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# поиск макс. длины токена
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# подгон токенов под одну длинну
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создание маски
attention_mask = np.where(padded != 0, 1, 0)

Инициализируем конфигурацию _BertConfig_. В качестве аргумента передадим ей JSON-файл с описанием настроек модели. JSON (англ. JavaScript Object Notation, «объектная запись JavaScript») — это организованный по ключам поток цифр, букв, двоеточий и фигурных скобок, который возвращает сервер при запросе.

In [4]:
# инициализируем конфигурацию BertConfig
config = transformers.BertConfig.from_json_file('/datasets/ds_bert/bert_config.json')

# инициализируем саму модель класса BertModel + файл с предобученной моделью и конфигурацией
model = transformers.BertModel.from_pretrained('/datasets/ds_bert/rubert_model.bin', config=config)

Some weights of the model checkpoint at /datasets/ds_bert/rubert_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Эмбеддинги модель _BERT_ создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча = 100.

Преобразуем данные в формат тензоров (англ. tensor) — многомерных векторов в библиотеке torch. Тип данных _LongTensor_ (англ. «длинный тензор») хранит числа в «длинном формате», то есть выделяет на каждое число 64 бита.

Чтобы получить эмбеддинги для батча, передадим модели данные и маску `attention_mask_batch`.

Для ускорения вычисления функцией `no_grad()` в библиотеке **torch** укажем, что градиенты не нужны: модель _BERT_ обучать не будем.

In [5]:
# кодирование слов в векторы (энбеддинги)
batch_size = 100
embeddings = []

for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

Соберём все эмбеддинги в матрицу признаков вызовом функции `concatenate()`.

In [6]:
# выделение признаков
features = np.concatenate(embeddings)
target = df_tweets['positive']

# обучение и протестирование модель
LR_model = LogisticRegression()
LR_model.fit(features, target)
prediction = LR_model.predict(features)
print(accuracy_score(target, prediction))

1.0


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
display(pd.DataFrame(features))

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.253247,-0.219286,-1.608447,-0.502738,0.725892,0.245856,-0.220571,0.144129,0.218670,-0.914865,...,0.303589,0.092892,-0.837715,-0.152068,-0.508317,0.193780,0.232059,-0.492279,1.024564,-0.411419
1,-0.233973,0.228894,-0.788425,-0.680701,0.537587,0.969870,-0.081464,0.394526,0.076317,0.116301,...,0.945812,-0.374136,-0.435543,-0.393317,-0.130384,-1.257408,-0.835614,-0.602395,0.494750,-0.903654
2,-0.082147,0.442392,-0.220233,-0.337405,0.514747,-0.049011,-0.268052,-0.078710,0.076074,0.290851,...,-0.024144,0.115906,-1.464156,-0.638169,0.176227,-0.237281,-0.016916,0.300110,1.012946,-0.370803
3,-0.884442,-0.611157,-0.552049,-0.646139,0.612808,0.250996,-0.445902,0.698740,0.267339,0.180597,...,0.400547,1.293199,-1.793786,-1.178009,0.280816,-0.385244,-0.009727,-1.077435,0.979432,-1.021986
4,0.469277,-0.231216,-0.678387,0.047896,0.702712,0.460773,0.433854,-0.117043,0.168424,-0.219428,...,0.433907,0.351283,-0.542122,-0.533007,0.223880,0.262744,0.009598,0.017871,0.513239,-0.188388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.023489,0.009346,-0.728182,0.085365,0.021931,0.255060,-0.047340,-0.094749,0.247499,0.028535,...,-0.045042,-0.016276,-0.386826,-0.004383,-0.183166,-0.237796,-1.104481,-0.016082,0.625905,-0.177017
396,0.037798,-0.023409,-0.508086,-0.082595,0.517871,0.254651,-0.199214,0.004147,0.518994,-0.020038,...,0.076869,-0.470810,-0.617664,0.140889,0.353920,-0.506663,-1.058230,0.026615,0.400752,-0.025611
397,-1.397274,-0.383809,-0.738730,0.155169,1.873098,-0.052520,-0.381069,0.653646,-0.437332,1.328871,...,-0.253383,1.636567,-1.240876,-0.476979,0.727108,-0.812097,-0.800558,-1.010794,0.252874,-1.172189
398,-1.131692,-0.225830,-0.161463,-0.879736,0.015725,1.093419,-1.050108,0.848875,0.233880,0.321800,...,0.105072,0.362218,-0.497042,-0.819721,0.225314,-0.611872,-1.117782,-0.538984,0.530190,-0.673876
